In [ ]:
import soccerdata as sd
import matplotlib.pyplot as plt
import sklearn as skl
import pandas as pd
import numpy as np
import pyarrow
import seaborn as sb
import csv
import os
from mplsoccer import VerticalPitch, Pitch
import math

In [ ]:
DIR = 'utd_games'
num = sum(os.path.isfile(os.path.join(DIR, name)) for name in os.listdir(DIR))
num

In [ ]:
games = []
for i in range(num):
    games.append(os.listdir(DIR)[i])

In [ ]:
game = pd.read_csv(DIR+'/'+games[0])

In [ ]:
game.head()

In [ ]:
def find_opp(df):
    teams = df['team'].sort_values().to_list()
    opposition = ''
    if teams[0] == 'Man Utd':
        print('Man Utd = teams[0]')
        return teams[-1]
    else:
        return  teams[0]
    print('Man Utd = teams[-1]')

In [ ]:
pass_df = game[['team','player','player_id','minute','type','outcome_type','second','x', 'y','end_x','end_y',]]

opposition = find_opp(pass_df)
mutd_df = pass_df[pass_df['team'] == 'Man Utd']
opposition_df = pass_df[pass_df['team'] == opposition]

In [ ]:
def splitter(df):
    return_df = df[(df['type'] == 'Pass') & (df['outcome_type'] == 'Successful')]
    return_df['x']*=1.2
    return_df['end_x']*=1.2
    return_df['y']*=0.8
    return_df['end_y']*=0.8
    return_df.insert(1, "pass_angle",
                      np.degrees(np.arctan2(
                        return_df['end_y'] - return_df['y'],
                        return_df['end_x'] - return_df['x']
                      ))
                     )
    return_df.insert(2, "pass_angle_abs",
                     np.abs(return_df['pass_angle'])
                     )
    return_df.insert(3, 'recevier',
                     return_df['player_id'].shift(-1)
                     )
    return return_df

In [46]:
def pass_comb(df):
    df['pos_min'] = df[['player_id', 'receiver']].min(axis = 1)
    df['pos_max'] = df[['player_id', 'receiver']].min(axis = 1)

    passimg_comb = df.groupby(['pos_min', 'pos_max']).size().reset_index(name='pass_count')

    return passimg_comb

In [ ]:
def enhance_position(df):
    avg_locs = df.groupby('player_id').agg({
        'x': 'median',
        'y': 'median',
        'player_id': 'count'
    })
    avg_locs.columns = ['x_avg', 'y_avg', 'pass_count']
    player_info = {}
    for player in team_players:
        player_id = player['playerId']
        player_name = player_names_dict.get(str(player_id), player['name'])
        player_info[player_id] = {
            'name': player_name,
            'shirtNo': player['shirtNo'],
            'position': player['position'],
            'isFirstEleven': player.get('isFirstEleven', False)
        }
    avg_locs = avg_locs.join(player_df)

    return avg_locs

In [ ]:
mutd_pass_df = splitter(mutd_df)
opposition_pass_df = splitter(opposition_df)

In [ ]:
mutd_pass_df

In [ ]:
scatter_df = pd.DataFrame()
for i, name in enumerate(mutd_pass_df["player"].unique()):
    passx = mutd_pass_df.loc[mutd_pass_df["player"] == name]["x"].to_numpy()
    recx = mutd_pass_df.loc[mutd_pass_df["pass"] == name]["end_x"].to_numpy()
    passy = mutd_pass_df.loc[mutd_pass_df["player"] == name]["y"].to_numpy()
    recy = mutd_pass_df.loc[mutd_pass_df["pass"] == name]["end_y"].to_numpy()
    scatter_df.at[i, "player"] = name

    scatter_df.at[i, "x"] = np.mean(np.concatenate([passx, recx]))
    scatter_df.at[i, "y"] = np.mean(np.concatenate([passy, recy]))

scatter_df['marker_size'] = (scatter_df['no'] / scatter_df['no'].max() * 1500)